In [53]:
import pandas as pd
from tqdm import tqdm

In [46]:
gpt4_secret_key = ""

In [47]:
from openai import OpenAI
client = OpenAI(api_key=gpt4_secret_key)

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

In [51]:
# response.choices[0].message.content

'The 2020 World Series was played in Arlington, Texas at the Globe Life Field, which was the new home stadium for the Texas Rangers.'

In [ ]:
result_path = '../data/results.csv'
result = pd.read_csv(result_path, index_col=0)
result

,Word,Meaning,Examples/0,Examples/1,Results
0,bilip,to change or influence,if the boy had known skipping class would nega...,gail’s parents limit her television time becau...,to affect or affect
1,derst,to be of advantage or use,"stranded in the desert, jake tried to restart ...",the medicine will not derst the patient if it ...,to cause something to be taken
2,ician,detailed and careful,a ician investigation of the crime brought out...,the detailer is always ician when cleaning my ...,able to be done with precision
3,redis,conduct,"explaining that his redis was unacceptable, th...",rewards for good redis helped the wayward chil...,a good attitude or attitude
4,celint,to give up or give in,"although jerry gave up smoking, there is alway...",sarah is so vain she thinks every man will cel...,to change something or something
...,...,...,...,...,...
1271,thwor,the size or length of something,the seamstress guessed the thwor of the man’s ...,"because the thwor of the shelf was off, it cou...",the length of something that is a part of some...
1272,enesis,to clarify something so it can be understood,the police officer could not enesisthe deaf wo...,"on the trip, we needed a guide to enesisthe fo...",to speak or speak something
1273,mulosy,the rate or speed of music or some other activity,"speeding up the mulosy, the street musician at...","keeping mulosy with the drummer, the singer ma...",a song that is played faster or faster
1274,rously,a large area where crops are grown,the slaves labored on the cotton rously while ...,if the rously owner cannot find laborers to pi...,a cotton cotton slack that is used to make a h...


In [37]:
def f_instr(ground_truth, pred):
    return f"""
The following is a ground truth definition and a ChatGPT-like systems’ outputs. Please rate an overall score on a
ten-point scale of the similarity of the output to the ground truth and give explanations to justify your scores.
Ground Truth:
{ground_truth}

System:
{pred}

Output in the format:
Score: x/10
Reasoning: ...
"""

In [38]:
# iterate over entries in results and create an instruction for each
instructions = []
for i, row in result.iterrows():
    instructions.append(f_instr(row['Meaning'], row['Results']))

In [55]:
# gpt4 eval response for each instruction
gpt4_responses = []
for i, instr in tqdm(enumerate(instructions)):
    response = client.chat.completions.create(
      model="gpt-4",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instr},
      ]
    )
    gpt4_responses.append(response.choices[0].message)

1276it [1:02:23,  2.93s/it]


In [61]:
# update result df with eval results
result['gpt4_eval'] = [repr(gpt4_responses[i].content) for i in range(len(gpt4_responses))]

In [62]:
# write to csv
result.to_csv("../out/t5_eval.csv")